In [1]:
import pandas as pd
import re
import underthesea
import os

In [2]:
data_path = "./Dataset_For_Work"

In [3]:
def load_data(file_path, label):
    """
    Đọc dữ liệu từ file và gắn nhãn tương ứng.
    Args:
        file_path (str): Đường dẫn đến tệp văn bản.
        label (str): Nhãn cảm xúc (positive, neutral, negative).
    Returns:
        data (list): Danh sách chứa các câu và nhãn.
    """
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            sentence = line.strip()  # Xóa khoảng trắng ở đầu và cuối câu
            if sentence:  # Chỉ thêm những câu không rỗng
                data.append((sentence, label))
    return data

In [4]:
def clean_text(text):
    """
    Làm sạch văn bản bằng cách loại bỏ ký tự đặc biệt, số và khoảng trắng thừa.
    Args:
        text (str): Câu văn bản cần làm sạch.
    Returns:
        str: Văn bản sau khi làm sạch.
    """
    text = text.lower()  # Chuyển thành chữ thường
    text = re.sub(r'\d+', '', text)  # Loại bỏ số
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text).strip()  # Loại bỏ khoảng trắng thừa
    return text

In [5]:
# Đọc dữ liệu từ các tệp và gắn nhãn
positive_data = load_data(os.path.join(data_path, "SA-training_positive.txt"), "positive")
neutral_data = load_data(os.path.join(data_path, "SA-training_neutral.txt"), "neutral")
negative_data = load_data(os.path.join(data_path, "SA-training_negative.txt"), "negative")

# Kết hợp tất cả dữ liệu
all_data = positive_data + neutral_data + negative_data

# Tiền xử lý văn bản
cleaned_data = [(clean_text(sentence), label) for sentence, label in all_data]

# Chuyển dữ liệu sang DataFrame
df = pd.DataFrame(cleaned_data, columns=["Sentence", "Label"])

# Hiển thị một vài dòng đầu tiên
print(df.head())

                                            Sentence     Label
0  đang xài mx dùng bình thường ngon pin trâu mỗi...  positive
1  qủa pin ngon sạc lại được bền riêng em dùng pi...  positive
2  cũng đang xài con logitech bluetooth tầm thấp ...  positive
3  logitech pin trâu thôi rôi mua con b cui ma cu...  positive
4  em có con chuột không dây k cũng đầy đủ nút bấ...  positive


In [6]:
from underthesea import word_tokenize

# Tải file stopwords
stop_words_path = "./Dataset_For_Work/vietnamese-stopwords.txt"

with open(stop_words_path, 'r', encoding='utf-8') as f:
    stop_words = set(f.read().splitlines())

# Loại bỏ stopwords
def remove_stopwords(sentence):
    # Tokenize and filter stopwords
    word_tokens = word_tokenize(sentence)
    filtered_words = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Thay mác label
def replace_labels(df, label_column):
    sentiment_map = {'positive': 1, 'negative': -1, 'neutral': 0}
    if label_column in df.columns:
        df[label_column] = df[label_column].map(sentiment_map)

df['Sentence'] = df['Sentence'].apply(remove_stopwords)
replace_labels(df, 'label')

# Cập nhật dữ liệu
print("Dữ liệu sau khi xử lý:")
print(df.head())

Dữ liệu sau khi xử lý:
                                            Sentence     Label
0  xài mx bình thường ngon pin trâu tội thằng chỗ...  positive
1               pin ngon sạc bền pin aa aaa thay thế  positive
2      xài logitech bluetooth tầm m xài đc ngon chán  positive
3  logitech pin trâu thôi rôi mua b cui ma cuc pi...  positive
4                chuột dây k đầy đủ nút bấm pin thay  positive


In [7]:
# Thực hiện word tokenization
def tokenize_text(sentence):
    """
    Tách từ cho câu đã được tiền xử lý.
    Args:
        sentence (str): Chuỗi văn bản cần tách từ.
    Returns:
        str: Văn bản sau khi tách từ.
    """
    return ' '.join(word_tokenize(sentence))  # Tách từ và kết hợp lại thành chuỗi

# Thực hiện tách từ cho từng câu trong cột "Sentence"
df['Sentence'] = df['Sentence'].apply(tokenize_text)

# Hiển thị dữ liệu sau khi tách từ
print("Dữ liệu sau khi tách từ:")
print(df.head(10))

Dữ liệu sau khi tách từ:
                                            Sentence     Label
0  xài mx bình thường ngon pin trâu tội thằng chỗ...  positive
1               pin ngon sạc bền pin aa aaa thay thế  positive
2      xài logitech bluetooth tầm m xài đc ngon chán  positive
3  logitech pin trâu thôi rôi mua b cui ma cuc pi...  positive
4                chuột dây k đầy đủ nút bấm pin thay  positive
5  v bluetooth trâu kém cạnh cạnh bấm chuột logit...  positive
6                                            sài cực  positive
7                              lắm ko mx anywhere ổn  positive
8  dragon war g thor xài ngon build cao cấp lắm đ...  positive
9  người thân chuột lazer logitech xài rùi hư ko ...  positive


In [8]:
from gensim.models import Word2Vec
from underthesea import word_tokenize
import pandas as pd

In [9]:
# Chuẩn bị dữ liệu cho Word2Vec (danh sách các câu dưới dạng tokenized)
tokenized_sentences = [sentence.split() for sentence in df['Sentence']]

In [10]:
# Huấn luyện mô hình Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_sentences,
                          vector_size=100,  # Kích thước vector (có thể tùy chỉnh)
                          window=5,         # Kích thước cửa sổ ngữ cảnh
                          min_count=2,      # Chỉ học từ xuất hiện >= 2 lần
                          workers=4,        # Số luồng CPU sử dụng
                          sg=1)             # Sử dụng Skip-Gram (sg=1) hoặc CBOW (sg=0)

# Lưu mô hình
word2vec_model.save("word2vec_vi.model")

In [11]:
#Lấy và kiểm tra vector của một từ cụ thể
word = "tốt"
if word in word2vec_model.wv:
    print(f"Vector của từ '{word}':\n{word2vec_model.wv[word]}")
else:
    print(f"Từ '{word}' không có trong vocabulary của Word2Vec.")

Vector của từ 'tốt':
[ 1.31314456e-01  1.75479367e-01  1.18545201e-02 -2.89897881e-02
  5.81628904e-02 -3.00739586e-01  1.07276045e-01  1.96083814e-01
 -1.63610324e-01 -9.20557678e-02 -1.67523250e-01 -2.52700388e-01
 -4.10921276e-02  5.36708198e-02  8.24685916e-02 -1.77998483e-01
 -1.32612372e-02 -2.44665459e-01 -2.44960375e-02 -4.33506578e-01
 -1.41073149e-02  1.28840312e-01  1.00705646e-01 -1.82382856e-02
 -2.36744676e-02 -4.52114716e-02 -1.00837104e-01 -1.74277976e-01
 -2.63296336e-01  1.62277177e-01  2.12181270e-01  2.60366332e-02
  2.17959024e-02 -1.38788819e-01 -1.75577760e-01  2.82581538e-01
  1.17195323e-02 -1.07001312e-01 -1.38254046e-01 -3.02115321e-01
  4.12724242e-02 -1.60955444e-01 -9.79643166e-02  1.25913501e-01
  1.68812722e-01 -1.95110887e-01 -2.09693506e-01 -8.17426667e-02
  1.26047045e-01  5.49336262e-02  1.25312462e-01 -2.10658193e-01
 -4.01224121e-02 -7.24987686e-02 -1.21316977e-01  1.63630053e-01
  7.37201348e-02 -8.60384777e-02 -1.95207760e-01 -7.31141716e-02
  1.

In [12]:
# Biểu diễn câu bằng Word2Vec
import numpy as np

def sentence_vector(sentence, model):
    """
    Tính trung bình vector của các từ trong câu.
    Args:
        sentence (str): Câu đã tách từ.
        model: Mô hình Word2Vec đã huấn luyện.
    Returns:
        np.array: Vector biểu diễn của câu.
    """
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  # Trả về vector 0 nếu không có từ nào hợp lệ
    return np.mean(word_vectors, axis=0)

# Thêm cột vector cho từng câu
df['Sentence_Vector'] = df['Sentence'].apply(lambda x: sentence_vector(x, word2vec_model))

# Hiển thị kết quả
print("Biểu diễn vector của các câu:")
print(df.head(10))


Biểu diễn vector của các câu:
                                            Sentence     Label  \
0  xài mx bình thường ngon pin trâu tội thằng chỗ...  positive   
1               pin ngon sạc bền pin aa aaa thay thế  positive   
2      xài logitech bluetooth tầm m xài đc ngon chán  positive   
3  logitech pin trâu thôi rôi mua b cui ma cuc pi...  positive   
4                chuột dây k đầy đủ nút bấm pin thay  positive   
5  v bluetooth trâu kém cạnh cạnh bấm chuột logit...  positive   
6                                            sài cực  positive   
7                              lắm ko mx anywhere ổn  positive   
8  dragon war g thor xài ngon build cao cấp lắm đ...  positive   
9  người thân chuột lazer logitech xài rùi hư ko ...  positive   

                                     Sentence_Vector  
0  [0.06486613, 0.18778491, 0.0515426, 0.00920007...  
1  [0.12164089, 0.24702281, 0.015478288, 0.037571...  
2  [0.037774615, 0.2879058, 0.12895194, 0.0463896...  
3  [-0.045497533, 0.145

In [13]:
# Kiểm tra sự tương đồng giữa các từ
# Tìm các từ tương tự với từ 'tốt'
word = "tốt"
if word in word2vec_model.wv:
    print(f"Các từ tương tự với '{word}':")
    similar_words = word2vec_model.wv.most_similar(word, topn=5)
    for similar_word, score in similar_words:
        print(f"{similar_word}: {score:.4f}")
else:
    print(f"Từ '{word}' không có trong từ điển của mô hình.")

Các từ tương tự với 'tốt':
mát: 0.9860
nhầm: 0.9832
dạng: 0.9820
nắng: 0.9819
nhược: 0.9813


In [14]:
# Tính khoảng cách cosine giữa hai từ
word1 = "tốt"
word2 = "xấu"

if word1 in word2vec_model.wv and word2 in word2vec_model.wv:
    similarity = word2vec_model.wv.similarity(word1, word2)
    print(f"Độ tương đồng cosine giữa '{word1}' và '{word2}': {similarity:.4f}")
else:
    print(f"'{word1}' hoặc '{word2}' không có trong từ điển của mô hình.")


Độ tương đồng cosine giữa 'tốt' và 'xấu': 0.7479


In [15]:
# Thử nghiệm bài toán Word Analogy
positive_words = ["đẹp", "xấu"]
negative_words = ["tốt"]

if all(word in word2vec_model.wv for word in positive_words + negative_words):
    result = word2vec_model.wv.most_similar(positive=positive_words, negative=negative_words, topn=1)
    print(f"Kết quả Word Analogy ({positive_words} - {negative_words}): {result}")
else:
    print("Một trong các từ không có trong từ điển của mô hình.")

Kết quả Word Analogy (['đẹp', 'xấu'] - ['tốt']): [('kế', 0.7815371155738831)]


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [18]:
# Chuẩn bị dữ liệu
X = np.vstack(df['Sentence_Vector'].values)  # Ma trận vector biểu diễn câu
y = df['Label'].values  # Nhãn cảm xúc

# Tách dữ liệu train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình học máy
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = clf.predict(X_test)
print("Độ chính xác trên tập test:", accuracy_score(y_test, y_pred))
print("Báo cáo phân loại:\n", classification_report(y_test, y_pred))

Độ chính xác trên tập test: 0.49411764705882355
Báo cáo phân loại:
               precision    recall  f1-score   support

    negative       0.44      0.42      0.43       335
     neutral       0.44      0.53      0.48       327
    positive       0.61      0.53      0.57       358

    accuracy                           0.49      1020
   macro avg       0.50      0.49      0.49      1020
weighted avg       0.50      0.49      0.50      1020

